### Imports and Initialization

In [1]:
# imports 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import pickle
import random
import pandas as pd
import numpy as np
import math
!pip install skorch
from skorch import NeuralNet
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import time
from forecast_evaluation import R2OS, CW_test, MSE, MAE, R2LOG, DM_test
from sklearn import metrics

display = ['out-of-sample R2', 'MSPE_bmk', 'losses', 'CW_test', 'elapsed time']

In [2]:
class StockDataset(Dataset):
    
    def __init__(self, x, y, window_size):
        
        self.window_size = window_size
        x = self.tssplit(x)
        self.x = torch.from_numpy(x).type(torch.float32)
        self.y = torch.from_numpy(y).type(torch.float32)
        self.length = x.shape[0]
        
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y
        
    def __len__(self):
        return self.length
    
    def tssplit(self, data):
        window_size = self.window_size
        X = np.zeros((data.shape[0]-window_size, window_size, data.shape[1]))
        for i in range(data.shape[0]-window_size):
            X[i,:] = data[i:i+window_size,:]
        return X

In [3]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.droput = dropout
        # input_size shows the number of features for each timestep.
        # we don't have to explicitly say how many sequences we'll have
        # input of RNN: #batches  x time_seq x # n_features
        self.rnn = nn.RNN(input_size,
                          hidden_size,
                          num_layers,
                          batch_first=True,
                          dropout=dropout)
        # we will concatenate all sequences from each timestep and send it
        # to the linear layer.
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        # initialize hidden state
        # x.size(0) is the batch size
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # we don't need to store hidden state
        out, _ = self.rnn(x, h0)
        
        # pass all training examples, last hidden state, all features
        out = self.fc(out[:, -1, :])
        return out

    
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.droput = dropout
        # input_size shows the number of features for each timestep.
        # we don't have to explicitly say how many sequences we'll have
        # input of RNN: #batches  x time_seq x # n_features
        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            batch_first=True,
                            dropout=dropout)
        # we will concatenate all sequences from each timestep and send it
        # to the linear layer.
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        # initialize hidden state
        # x.size(0) is the batch size
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # we don't need to store hidden state
        out, _ = self.lstm(x, (h0, c0))
        
        # pass all training examples, last hidden state, all features
        out = self.fc(out[:, -1, :])
        return out
    

class GRU(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.droput = dropout
        # input_size shows the number of features for each timestep.
        # we don't have to explicitly say how many sequences we'll have
        # input of RNN: #batches  x time_seq x # n_features
        self.rnn = nn.RNN(input_size,
                          hidden_size,
                          num_layers,
                          batch_first=True,
                          dropout=dropout)
        # we will concatenate all sequences from each timestep and send it
        # to the linear layer.
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        # initialize hidden state
        # x.size(0) is the batch size
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # we don't need to store hidden state
        out, _ = self.rnn(x, h0)
        
        # pass all training examples, last hidden state, all features
        out = self.fc(out[:, -1, :])
        return out

In [4]:
def expanding_window_recurrent_skorch(x, y, test_start, window_size, cv, model_args, cv_args, workers=-1,
                              verbosity=10, random_seeds=None, parallel=True, standardize=True, retrain_interval=1):
    """
    DESCRIPTION
    -----------
    Produce 1-step ahead forecasts, trained with recursive expanding windows.
    Uses parallelization on multiple CPU cores to speed up the process.

    PARAMETERS
    -----------
    X_train: 2-dimensional numpy array, independent variables used to train the forecast model

    X_test: 2-dimensional numpy array, independent variables used for forecasting

    y_train: 1-dimensional numpy array, dependent variable used to train the forecast model

    y_test: 1-dimensional numpy array, dependent variable to forecast

    forecast_model: python object, model used for forecasting. Make this object similar to the

    sklearn models, with a fit() and predict() function

    model_args: dict, arguments to pass when initializing the model

    workers: int, how many processes are used. -1 denotes using all available logical processors
    """

    start_time = time.time()

    # if no model arguments, parse an empty dictionary
    if model_args is None:
        model_args = {}

    n_periods = int((x.shape[0] - test_start) / retrain_interval)
    pred_model = np.zeros((n_periods*retrain_interval,))
    pred_bmk = np.zeros((n_periods*retrain_interval,))

    if random_seeds is not None:
        if len(random_seeds) != n_periods:
            raise ValueError('length of random_seeds must be same as length of X_test')
            
    if (x.shape[0] - test_start) % retrain_interval != 0:
        raise ValueError('number of forecast periods is not divisible by retrain_intervals')

    # function to produce recursive expanding window forecasts
    def ExpW_forecast(x, y, i, test_start):

        if random_seeds is not None:
            torch.manual_seed(random_seeds[i])
            random.seed(random_seeds[i])
            np.random.seed(random_seeds[i])

        # ensure safety of x and y matrices
        x = x.copy()
        y = y.copy()
        
        # historical average forecast as the baseline
        bmk = np.zeros((retrain_interval,1))
        for j in range(retrain_interval):
            bmk[j] = y[:test_start-window_size+i*retrain_interval+j].mean()
            
        x_std = x[0:test_start+i*retrain_interval]
        y_std = y[0:test_start+i*retrain_interval]

        if standardize:
            # scale the data
            x_scaler = StandardScaler().fit(x_std)
            y_scaler = StandardScaler().fit(y_std)

            x = x_scaler.transform(x)        
            y = y_scaler.transform(y)
        
        dataset = StockDataset(x, y, window_size=window_size)
        
        # take the correct subsets of the dataset for training and testing
        fit_subset = Subset(dataset, range(0,test_start-window_size+i*retrain_interval))
        pred_subset = Subset(dataset, range(test_start-window_size+i*retrain_interval,test_start-window_size+(i+1)*retrain_interval))

        # forecasts of model
        forecast_model = NeuralNet(**model_args)
        model = GridSearchCV(forecast_model, cv_args, refit=True, cv=cv, scoring='neg_mean_squared_error')
        model = model.fit(fit_subset[:][0],fit_subset[:][1])
        pred = model.predict(pred_subset).reshape(-1,1)
            
        if not parallel:
            print(f'{i + 1}/{n_periods}', end='\r')
        
        # return a tuple of the baseline and the expanding window forecasts
        if standardize:
            out = bmk, y_scaler.inverse_transform(pred)
        else:
            out = bmk, pred

        return out

    # run the forecast models in parallel
    if parallel:
        pred_pairs = Parallel(n_jobs=workers, backend='loky', verbose=verbosity) \
            (delayed(ExpW_forecast)(x, y, i, test_start) for i in range(n_periods))
            
    else:
        pred_pairs = [ExpW_forecast(x, y, i, test_start) for i in range(n_periods)]

    # split the prediction tuples
    for i, pred_pair in enumerate(pred_pairs):
        pred_bmk[i*retrain_interval:(i+1)*retrain_interval] = pred_pair[0].flatten()
        pred_model[i*retrain_interval:(i+1)*retrain_interval] = pred_pair[1].flatten()

    # calculate metrics
    y_test = y[test_start:]
    MSPE_model = metrics.mean_squared_error(y_test, pred_model)
    MSPE_bmk = metrics.mean_squared_error(y_test, pred_bmk)
    R2OS_val = R2OS(MSPE_model, MSPE_bmk)
    cw_test= CW_test(y_test, pred_bmk, pred_model)
    losses = {
        'MSE': MSE(y_test, pred_model),
        'MAE': MAE(y_test, pred_model),
        'R2LOG': R2LOG(y_test, pred_model)
    }


    end_time = time.time()
    elapsed_time = end_time - start_time
    
    out = {'out-of-sample R2': R2OS_val,
            'MSPE_bmk': MSPE_bmk,
            'MSPE_model': MSPE_model,
            'pred_bmk': pred_bmk,
            'pred_model': pred_model,
            'losses': losses,
            'CW_test': cw_test,
            'elapsed time': elapsed_time}
    
    return out

### ========================================================= ###


data_stocks = pd.read_csv('data_stocks.csv')
x = np.array(data_stocks.drop(columns=['yyyymm', 'EQPREM']), dtype=np.float32)
y = np.array(data_stocks.loc[:,['EQPREM']], dtype=np.float32)
# ADD PAST EQPREM TO FEATURE MATRIX
x = np.hstack([x, y])

display = ['out-of-sample R2', 'MSPE_bmk', 'losses', 'CW_test', 'elapsed time']

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

'''

model_args = {
    'module': RNN,
    'module__input_size': 13,
    'module__hidden_size': 10,
    'module__num_layers': 2,
    'module__dropout': 0.5,
    'criterion': torch.nn.MSELoss,
    'optimizer': optim.SGD,
    'max_epochs': 10,
    'batch_size': 36,
    'train_split': None,
    'verbose': 0
}

cv_args = {
    'module__hidden_size':[10],
    'module__num_layers':[2]
}

results = expanding_window_recurrent_skorch(x[0:408], 
                                         y[0:408], 
                                         360,
                                         window_size=12,
                                         cv=TimeSeriesSplit(2),
                                         model_args=model_args,
                                         cv_args=cv_args,
                                         random_seeds=None,
                                         verbosity=50,
                                         workers=-1,
                                         parallel=False,
                                         standardize=False,
                                         retrain_interval=12)

for item in display:
    print(f'{item}: {results[item]}')
    
results['pred_model']
'''

"\n\nmodel_args = {\n    'module': RNN,\n    'module__input_size': 13,\n    'module__hidden_size': 10,\n    'module__num_layers': 2,\n    'module__dropout': 0.5,\n    'criterion': torch.nn.MSELoss,\n    'optimizer': optim.SGD,\n    'max_epochs': 10,\n    'batch_size': 36,\n    'train_split': None,\n    'verbose': 0\n}\n\ncv_args = {\n    'module__hidden_size':[10],\n    'module__num_layers':[2]\n}\n\nresults = expanding_window_recurrent_skorch(x[0:408], \n                                         y[0:408], \n                                         360,\n                                         window_size=12,\n                                         cv=TimeSeriesSplit(2),\n                                         model_args=model_args,\n                                         cv_args=cv_args,\n                                         random_seeds=None,\n                                         verbosity=50,\n                                         workers=-1,\n                      

### Evaluate Forecasts

In [8]:
model_args = {
    'module': RNN,
    'module__input_size': 13,
    'module__hidden_size': 10,
    'module__num_layers': 2,
    'module__dropout': 0.1,
    'criterion': torch.nn.MSELoss,
    'optimizer': optim.SGD,
    'max_epochs': 100,
    'batch_size': 36,
    'train_split': None,
    'verbose': 0
}

cv_args = {
    'module__hidden_size':[10, 100],
    'module__num_layers':[2, 3],
    'lr': [0.1, 0.001]
}

results = expanding_window_recurrent_skorch(x[0:1080], 
                                         y[0:1080], 
                                         360,
                                         window_size=12,
                                         cv=TimeSeriesSplit(3),
                                         model_args=model_args,
                                         cv_args=cv_args,
                                         random_seeds=range(60),
                                         verbosity=50,
                                         workers=-1,
                                         parallel=True,
                                         standardize=True,
                                         retrain_interval=12)

for item in display:
    print(f'{item}: {results[item]}')

with open('nn_results/results_RNN.pickle', 'wb') as file:
    pickle.dump(results, file)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

In [9]:
model_args = {
    'module': LSTM,
    'module__input_size': 13,
    'module__hidden_size': 10,
    'module__num_layers': 2,
    'module__dropout': 0.1,
    'criterion': torch.nn.MSELoss,
    'optimizer': optim.SGD,
    'max_epochs': 100,
    'batch_size': 36,
    'train_split': None,
    'verbose': 0
}

cv_args = {
    'module__hidden_size':[10, 100],
    'module__num_layers':[2, 3],
}


results = expanding_window_recurrent_skorch(x[0:1080], 
                                         y[0:1080], 
                                         360,
                                         window_size=12,
                                         cv=TimeSeriesSplit(3),
                                         model_args=model_args,
                                         cv_args=cv_args,
                                         random_seeds=range(60),
                                         verbosity=50,
                                         workers=-1,
                                         parallel=True,
                                         standardize=True,
                                         retrain_interval=12)

for item in display:
    print(f'{item}: {results[item]}')

with open('nn_results/results_LSTM.pickle', 'wb') as file:
    pickle.dump(results, file)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

In [10]:
model_args = {
    'module': GRU,
    'module__input_size': 13,
    'module__hidden_size': 10,
    'module__num_layers': 2,
    'module__dropout': 0.1,
    'criterion': torch.nn.MSELoss,
    'optimizer': optim.SGD,
    'max_epochs': 100,
    'batch_size': 36,
    'train_split': None,
    'verbose': 0
}

cv_args = {
    'module__hidden_size':[10, 100],
    'module__num_layers':[2, 3],
    'lr': [0.1, 0.001]
}


results = expanding_window_recurrent_skorch(x[0:1080], 
                                         y[0:1080], 
                                         360,
                                         window_size=12,
                                         cv=TimeSeriesSplit(3),
                                         model_args=model_args,
                                         cv_args=cv_args,
                                         random_seeds=range(60),
                                         verbosity=50,
                                         workers=-1,
                                         parallel=True,
                                         standardize=True,
                                         retrain_interval=12)

for item in display:
    print(f'{item}: {results[item]}')

with open('nn_results/results_GRU.pickle', 'wb') as file:
    pickle.dump(results, file)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

### DM Test

In [3]:
data_stocks = pd.read_csv('data_stocks.csv')

X = np.array(data_stocks.drop(columns=['yyyymm', 'EQPREM']))
y = np.array(data_stocks.loc[:,['EQPREM']]).flatten()

X_train = X[0:360,:]
X_test = X[360:1080,:]
y_train = y[0:360]
y_test = y[360:1080]


In [4]:
# RNN
filename = 'nn_results/results_RNN.pickle'
with open(filename, 'rb') as file:
    model_results =  pickle.load(file)
    pred_bmk = model_results['pred_bmk']
    pred_model = model_results['pred_model']
    results = DM_test(y_test, pred_bmk, pred_model)
    print(f'DM expw: {results}')


DM expw: -3.9526942101850637


In [5]:
# LSTM
filename = 'nn_results/results_LSTM.pickle'
with open(filename, 'rb') as file:
    model_results =  pickle.load(file)
    pred_bmk = model_results['pred_bmk']
    pred_model = model_results['pred_model']
    results = DM_test(y_test, pred_bmk, pred_model)
    print(f'DM expw: {results}')


DM expw: -2.5466903036473547


In [6]:
# GRU
filename = 'nn_results/results_LSTM.pickle'
with open(filename, 'rb') as file:
    model_results =  pickle.load(file)
    pred_bmk = model_results['pred_bmk']
    pred_model = model_results['pred_model']
    results = DM_test(y_test, pred_bmk, pred_model)
    print(f'DM expw: {results}')


DM expw: -2.5466903036473547
